In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
player_df = pd.read_csv('../data/new/players_all.csv')
player_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4742 entries, 0 to 4741
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Id        4742 non-null   int64 
 1   No.       4742 non-null   int64 
 2   Name      4742 non-null   object
 3   Age       4742 non-null   int64 
 4   Team      4742 non-null   object
 5   Position  4742 non-null   object
 6   year      4742 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 259.5+ KB


In [4]:
stats1_df = pd.read_csv('../data/new/1xbet_all.csv')
stats1_df = stats1_df.drop(['Team','Age','Position'], axis=1)
stats1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4756 entries, 0 to 4755
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      4756 non-null   object 
 1   Apps      4756 non-null   int64  
 2   Mins      4756 non-null   int64  
 3   Goals     4756 non-null   int64  
 4   Assists   4756 non-null   int64  
 5   SpG       4756 non-null   float64
 6   KeyP      4756 non-null   float64
 7   Drb_x     4756 non-null   float64
 8   Fouled    4756 non-null   float64
 9   Off       4756 non-null   float64
 10  Disp      4756 non-null   float64
 11  UnsTch    4756 non-null   float64
 12  Rating    4756 non-null   float64
 13  Tackles   4756 non-null   float64
 14  Inter     4756 non-null   float64
 15  Fouls     4756 non-null   float64
 16  Offsides  4756 non-null   float64
 17  Clear     4756 non-null   float64
 18  Drb_y     4756 non-null   float64
 19  Blocks    4756 non-null   float64
 20  year      4756 non-null   int6

In [5]:
stats2_df = pd.read_csv('../data/new/understat_all.csv')
stats2_df = stats2_df.drop(['No','Team'], axis=1)
stats2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4755 entries, 0 to 4754
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Player       4755 non-null   object 
 1   Apps         4755 non-null   int64  
 2   Min          4755 non-null   int64  
 3   G            4755 non-null   int64  
 4   NPG          4755 non-null   int64  
 5   A            4755 non-null   int64  
 6   xG           4755 non-null   float64
 7   NPxG         4755 non-null   float64
 8   xA           4755 non-null   float64
 9   xGChain      4755 non-null   float64
 10  xGBuildup    4755 non-null   float64
 11  xG90         4755 non-null   float64
 12  NPxG90       4755 non-null   float64
 13  xA90         4755 non-null   float64
 14  xG90+xA90    4755 non-null   float64
 15  NPxG90+xA90  4755 non-null   float64
 16  xGChain90    4755 non-null   float64
 17  xGBuildup90  4755 non-null   float64
 18  year         4755 non-null   int64  
dtypes: flo

In [6]:
salary_df = pd.read_csv('../data/new/salary_all.csv')
salary_df = salary_df.drop(['Position', 'Team'], axis=1)
salary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rank           4803 non-null   float64
 1   Name           4803 non-null   object 
 2   Weekly Salary  4803 non-null   int64  
 3   year           4803 non-null   int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 150.2+ KB


In [10]:
# player_df, salary_df, stats1_df, stats2_df
eda_df = player_df.merge(salary_df[['Weekly Salary','Name', 'year']], how='outer', left_on=['Name', 'year'], right_on=['Name', 'year'])
eda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6486 entries, 0 to 6485
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             4763 non-null   float64
 1   No.            4763 non-null   float64
 2   Name           6486 non-null   object 
 3   Age            4763 non-null   float64
 4   Team           4763 non-null   object 
 5   Position       4763 non-null   object 
 6   year           6486 non-null   int64  
 7   Weekly Salary  4856 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 405.5+ KB


In [15]:
eda_df.isnull().sum(axis=0)

Id               1723
No.              1723
Name                0
Age              1723
Team             1723
Position         1723
year                0
Weekly Salary    1630
dtype: int64

In [16]:
eda_df.isnull().mean()

Id               0.265649
No.              0.265649
Name             0.000000
Age              0.265649
Team             0.265649
Position         0.265649
year             0.000000
Weekly Salary    0.251311
dtype: float64

In [18]:
na_list = eda_df.loc[eda_df.isnull().any(axis=1)].loc[:, ['Name','year']]
na_list.groupby('year')['Name'].count()

year
2014    456
2015    443
2016    379
2017    300
2018    283
2019    387
2020    379
2021    361
2022    365
Name: Name, dtype: int64

In [19]:
player_df.groupby('year')['Name'].count()

year
2014    535
2015    547
2016    529
2017    516
2018    497
2019    511
2020    519
2021    533
2022    555
Name: Name, dtype: int64

In [22]:
na_list.index

Index([   5,    6,    7,   12,   13,   15,   18,   19,   20,   21,
       ...
       6476, 6477, 6478, 6479, 6480, 6481, 6482, 6483, 6484, 6485],
      dtype='int64', length=3353)

In [32]:
eda_df[eda_df['Weekly Salary'].isnull()].head()

,Id,No.,Name,Age,Team,Position,year,Weekly Salary
5,5.0,1185.0,Mile Jedinak,30.0,Crystal Palace,Midfielder,2014,NaN
6,6.0,1333.0,Phil Jones,22.0,Man Utd,Defender,2014,NaN
7,7.0,1201.0,Morgan Schneiderlin,24.0,Southampton,Midfielder,2014,NaN
12,12.0,1391.0,Robert Huth,30.0,Leicester,Defender,2014,NaN
13,498.0,1391.0,Robert Huth,30.0,Stoke,Defender,2014,NaN
